In [ ]:
! pip install transformers

In [1]:
# visualize_activations.py

import torch
import torch.nn.functional as F
import numpy as np
from transformers import RobertaTokenizerFast, RobertaModel
from modules import CBL, RobertaCBL
import config as CFG
import matplotlib.pyplot as plt
import streamlit as st

ModuleNotFoundError: No module named 'transformers'

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load tokenizer and models
tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
cbl = CBL(len(CFG.concept_set[args.dataset])).to(device)
cbl.load_state_dict(torch.load('cbl_path mpnet_acs/SetFit_sst2/roberta_cbm/cbl_acc_epoch_10.pt'))
cbl.eval()

NameError: name 'RobertaTokenizerFast' is not defined

In [ ]:
# Load the final predictor
final_predictor = torch.load('path_to_final_predictor.pt')

# Load concept list
concepts = CFG.concept_set[args.dataset]

# Function to get activations
def get_activations(input_text):
    inputs = tokenizer(input_text, return_tensors='pt', padding=True, truncation=True).to(device)
    with torch.no_grad():
        # Get embeddings from language model
        outputs = RobertaModel.from_pretrained('roberta-base').to(device)(**inputs)
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # CLS token

        # Get concept activations
        concept_activations = cbl(cls_embedding)
        concept_activations = F.relu(concept_activations)

        # Get prediction
        logits = final_predictor(concept_activations)
        prediction = torch.argmax(logits, dim=-1).item()

    activations = concept_activations.cpu().numpy().flatten()
    return activations, prediction

# Visualization
def visualize_activations(activations, concepts):
    # Sort concepts by activation
    sorted_indices = np.argsort(-activations)
    sorted_concepts = [concepts[i] for i in sorted_indices]
    sorted_activations = activations[sorted_indices]

    # Plot bar chart
    plt.figure(figsize=(10, 6))
    plt.barh(sorted_concepts[:10], sorted_activations[:10])  # Top 10 concepts
    plt.xlabel('Activation')
    plt.title('Top Activated Concepts')
    plt.gca().invert_yaxis()
    plt.show()

# Example usage
input_text = "The restaurant had great food but the service was terrible."
activations, prediction = get_activations(input_text)
visualize_activations(activations, concepts)
print("Prediction:", "Positive" if prediction == 1 else "Negative")
